<a href="https://colab.research.google.com/github/LithiraHettiarachchi/ForecastW/blob/main/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install -U scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pandas as pd
from scipy import stats
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression

In [29]:
load_data = pd.read_csv('/content/drive/MyDrive/ForestFire_Tracker/3361892.csv')

In [30]:
load_data

,STATION,NAME,DATE,AWND,PGTM,PRCP,TAVG,TMAX,TMIN,WDF2,WDF5,WSF2,WSF5,WT01,WT02,WT08
0,USW00093134,"LOS ANGELES DOWNTOWN USC, CA US",2022-01-01,1.12,NaN,0.00,NaN,62,45,260.0,260.0,6.0,10.1,NaN,NaN,1.0
1,USW00093134,"LOS ANGELES DOWNTOWN USC, CA US",2022-01-02,1.79,NaN,0.00,NaN,64,43,270.0,270.0,6.9,11.0,NaN,NaN,NaN
2,USW00093134,"LOS ANGELES DOWNTOWN USC, CA US",2022-01-03,1.34,NaN,0.00,NaN,62,42,270.0,270.0,6.9,11.0,NaN,NaN,NaN
3,USW00093134,"LOS ANGELES DOWNTOWN USC, CA US",2022-01-04,0.22,NaN,0.00,NaN,63,42,260.0,270.0,6.0,10.1,NaN,NaN,1.0
4,USW00093134,"LOS ANGELES DOWNTOWN USC, CA US",2022-01-05,0.22,NaN,0.00,NaN,69,43,270.0,260.0,6.0,8.1,1.0,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,USW00093134,"LOS ANGELES DOWNTOWN USC, CA US",2022-12-27,1.12,NaN,0.45,NaN,65,55,270.0,270.0,8.1,10.1,1.0,NaN,NaN
361,USW00093134,"LOS ANGELES DOWNTOWN USC, CA US",2022-12-28,0.89,NaN,0.01,NaN,64,51,140.0,270.0,6.9,10.1,1.0,NaN,1.0
362,USW00093134,"LOS ANGELES DOWNTOWN USC, CA US",2022-12-29,1.12,NaN,0.00,NaN,61,56,90.0,100.0,6.9,10.1,NaN,NaN,NaN
363,USW00093134,"LOS ANGELES DOWNTOWN USC, CA US",2022-12-30,0.45,NaN,0.01,NaN,60,56,100.0,80.0,4.9,6.9,1.0,NaN,NaN


In [31]:
null_vals = load_data.apply(pd.isnull).sum()/load_data.shape[0]
null_vals

STATION    0.000000
NAME       0.000000
DATE       0.000000
AWND       0.008219
PGTM       1.000000
PRCP       0.000000
TAVG       1.000000
TMAX       0.000000
TMIN       0.000000
WDF2       0.002740
WDF5       0.038356
WSF2       0.002740
WSF5       0.038356
WT01       0.712329
WT02       0.994521
WT08       0.586301
dtype: float64

In [32]:
valid_cols = load_data.columns[null_vals < .05]

In [27]:
valid_cols

Index(['STATION', 'NAME', 'DATE', 'AWND', 'PRCP', 'TMAX', 'TMIN', 'WDF2',
       'WDF5', 'WSF2', 'WSF5'],
      dtype='object')

In [28]:
load_data = [valid_cols].copy()

     X  Y  month  day  FFMC    DMC     DC   ISI  temp  RH  wind  rain   area
0    7  5      7    0  86.2   26.2   94.3   5.1   8.2  51   6.7   0.0   0.00
1    7  4     10    5  90.6   35.4  669.1   6.7  18.0  33   0.9   0.0   0.00
2    7  4     10    2  90.6   43.7  686.9   6.7  14.6  33   1.3   0.0   0.00
5    8  6      1    3  92.3   85.3  488.0  14.7  22.2  29   5.4   0.0   0.00
6    8  6      1    1  92.3   88.9  495.6   8.5  24.1  27   3.1   0.0   0.00
..  .. ..    ...  ...   ...    ...    ...   ...   ...  ..   ...   ...    ...
512  4  3      1    3  81.6   56.7  665.6   1.9  27.8  32   2.7   0.0   6.44
513  2  4      1    3  81.6   56.7  665.6   1.9  21.9  71   5.8   0.0  54.29
514  7  4      1    3  81.6   56.7  665.6   1.9  21.2  70   6.7   0.0  11.16
515  1  4      1    2  94.4  146.0  614.7  11.3  25.6  42   4.0   0.0   0.00
516  6  3      9    5  79.5    3.0  106.7   1.1  11.8  31   4.5   0.0   0.00

[497 rows x 13 columns]


<ipython-input-10-501ceda16e37>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['month'] = label_encoder.fit_transform(data['month'])
<ipython-input-10-501ceda16e37>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['day'] = label_encoder.fit_transform(data['day'])


In [ ]:

columns_to_normalize = ['FFMC', 'DMC', 'DC', 'ISI', 'temp', 'RH', 'wind', 'rain', 'area']
scaler = MinMaxScaler()
normalized_dataset = data.copy()
normalized_dataset[columns_to_normalize] = scaler.fit_transform(normalized_dataset[columns_to_normalize])
print(normalized_dataset)

     X  Y  month  day      FFMC       DMC        DC       ISI      temp  \
0    7  5      7    0  0.526066  0.080472  0.101325  0.186047  0.192926   
1    7  4     10    5  0.734597  0.112383  0.775419  0.260465  0.508039   
2    7  4     10    2  0.734597  0.141172  0.796294  0.260465  0.398714   
5    8  6      1    3  0.815166  0.285467  0.563035  0.632558  0.643087   
6    8  6      1    1  0.815166  0.297954  0.571948  0.344186  0.704180   
..  .. ..    ...  ...       ...       ...       ...       ...       ...   
512  4  3      1    3  0.308057  0.186264  0.771315  0.037209  0.823151   
513  2  4      1    3  0.308057  0.186264  0.771315  0.037209  0.633441   
514  7  4      1    3  0.308057  0.186264  0.771315  0.037209  0.610932   
515  1  4      1    2  0.914692  0.496011  0.711622  0.474419  0.752412   
516  6  3      9    5  0.208531  0.000000  0.115867  0.000000  0.308682   

           RH      wind  rain      area  
0    0.480000  0.741176   0.0  0.000000  
1    0.240000  

In [ ]:
X = normalized_dataset.drop('area', axis=1)
y = normalized_dataset['area']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = LinearRegression()


In [ ]:
model.fit(X_train, y_train)

LinearRegression()